# TODOs

- [x] Figure out why resumption is not working
- Verify that loss function is implemented right 
- record / plot loss better and easy background training
- [X] Research more QLoRA
  - [ ] make sure that QLoRA is actually happening..
  - RTFP
  - [X] Find good solid example code with transformers
  - Determine QLoRA parameters to experiment with and how
    - datatype
    - what bits
    - r
    - dropout
    - target_modules
    - lora alpha
- Determine training parameters to experiment with
  - AdamW RTFP
  - tune adamW starting learning rate?
  - Batch size. Should it vary over time?
  - 
- get complicated and weird
  - explore mining hard negatives
  - explore synthesizing hard negatives
  - synthesize more varied data
  - generated data off of a target domain (easy-on fine tuning)
- benchmark
  - add inferrence code path
  - figure out how to run model against an mteb benchmark
- update code to be able to run it distributed in the cloud?
  - related to deepspeed and accelerate
    - https://arxiv.org/abs/2104.07857
    - https://huggingface.co/docs/accelerate/usage_guides/deepspeed
- [x] get a bigger graphics card

In [2]:
%load_ext autoreload
%autoreload 2
seed = 42

In [13]:
import train

base_model, tokenizer = train.load_model()
model = train.load_model_for_training(base_model)

dataset = train.prepare_dataset(tokenizer, seed)
# Access the new train and test datasets
train_dataset = dataset["train"]
eval_dataset = dataset["test"]



In [14]:
# train_dataset[0]
tokenizer("foo bar baz!", add_special_tokens=True)

{'input_ids': [1, 7953, 2594, 12741, 29991, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:

from transformers import TrainingArguments, IntervalStrategy
from train import TinyEmbedTrainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    log_level="info",
    logging_strategy=IntervalStrategy.STEPS,
    gradient_checkpointing=True,
    logging_steps=5,
    save_steps=1000,
    seed=seed,
)

trainer = TinyEmbedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)
trainer.train(resume_from_checkpoint=False)

In [2]:
from experiments import experiment_quantization
import os

experiment_quantization.run()


/mnt/freezer/dev/tinyllama-embed/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-01-31 00:52:03,898] Using an existing study with name 'bits-quantization-5000' instead of creating a new one.


Using an existing study with name 'bits-quantization-5000' instead of creating a new one.


max_steps is given, it will override any value given in num_train_epochs
***** Running training *****
  Num examples = 163,636
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 100
  Number of trainable parameters = 5,902,336
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/mnt/freezer/dev/tinyllama-embed/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/mnt/freezer/dev/tinyllama-embed/.venv/lib/python3.10/sit

Step,Training Loss
5,0.658400
10,0.678400
15,0.696900
20,0.693900
25,0.663800
30,0.686200
35,0.654200
40,0.681800
45,0.637600
50,0.665200




Training completed. Do not forget to share your model on huggingface.co/models =)


[I 2024-01-31 00:55:36,865] Trial 5 finished with value: 0.6552062129974365 and parameters: {'r': 8, 'alpha': 64, 'dropout': 0.1, 'batch_size': 4, 'adam_beta1': 0.9, 'adam_beta2': 0.999, 'adam_epsilon': 1e-08, 'bits': 8}. Best is trial 0 with value: 0.26044891719818114.


Trial 5 finished with value: 0.6552062129974365 and parameters: {'r': 8, 'alpha': 64, 'dropout': 0.1, 'batch_size': 4, 'adam_beta1': 0.9, 'adam_beta2': 0.999, 'adam_epsilon': 1e-08, 'bits': 8}. Best is trial 0 with value: 0.26044891719818114.


loading configuration file config.json from cache at /mnt/freezer/huggingface-cache/hub/models--TinyLlama--TinyLlama-1.1B-intermediate-step-1195k-token-2.5T/snapshots/706bc2851338c4a89bb212e96ff23f9cc1ebde1d/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.37.2",
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file model.safetensors from cache at /mnt/freezer/huggingface-cache/hub/models--TinyLlama--TinyLlama-1.

Step,Training Loss
5,0.655000
10,0.682000
15,0.695400
20,0.692500
25,0.660800
30,0.687700
35,0.653000
40,0.668200
45,0.631600
50,0.663000


In [ ]:
"""
experiments:

Optimize first for quick tests, and tests that will make subsequent tests give better results and faster feedback

- 4 vs 8 bit loss, along with wall clock time 
- quick learning rate / alpha test to verify what achieves better loss over iterations
- optimize for loss:
  - batch size
  - lora r
  - dropout
  - combinations of target_modules???

"""
